In [19]:
from google.colab import files
uploaded = files.upload()

Saving olist_order_payments_dataset.csv to olist_order_payments_dataset.csv


In [20]:
import pandas as pd
import sqlite3


In [21]:
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")


In [22]:
# Convert timestamps
date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")


In [23]:
conn = sqlite3.connect("retail_oms.db")

orders.to_sql("orders", conn, index=False, if_exists="replace")
order_items.to_sql("order_items", conn, index=False, if_exists="replace")
customers.to_sql("customers", conn, index=False, if_exists="replace")
payments.to_sql("payments", conn, index=False, if_exists="replace")


103886

In [25]:
query = """
SELECT
    order_id,
    order_purchase_timestamp,
    order_delivered_customer_date,
    order_estimated_delivery_date,
    julianday(order_delivered_customer_date) -
    julianday(order_estimated_delivery_date) AS delay_days
FROM orders
WHERE order_status = 'delivered'
AND (
    julianday(order_delivered_customer_date) -
    julianday(order_estimated_delivery_date)
) > 0;
"""

delayed_orders = pd.read_sql(query, conn)
delayed_orders.head()


,order_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delay_days
0,203096f03d82e0dffbc41ebc2e2bcfb7,2017-09-18 14:31:30,2017-10-09 22:23:46,2017-09-28 00:00:00,11.933171
1,fbf9ac61453ac646ce8ad9783d7d0af6,2018-02-20 23:46:53,2018-03-21 22:03:54,2018-03-12 00:00:00,9.919375
2,8563039e855156e48fccee4d611a3196,2018-02-17 15:59:46,2018-03-20 00:59:25,2018-03-20 00:00:00,0.041262
3,6ea2f835b4556291ffdc53fa0b3b95e8,2017-11-24 21:27:48,2017-12-28 18:59:23,2017-12-21 00:00:00,7.791238
4,66e4624ae69e7dc89bd50222b59f581f,2018-03-09 14:50:15,2018-04-03 13:28:46,2018-04-02 00:00:00,1.561644


In [26]:
query = """
SELECT
    order_status,
    COUNT(*) AS total_orders
FROM orders
GROUP BY order_status;
"""

order_status_df = pd.read_sql(query, conn)
order_status_df


,order_status,total_orders
0,approved,2
1,canceled,625
2,created,5
3,delivered,96478
4,invoiced,314
5,processing,301
6,shipped,1107
7,unavailable,609


In [27]:
query = """
SELECT
    o.order_id,
    o.order_status,
    SUM(p.payment_value) AS total_payment
FROM orders o
JOIN payments p
ON o.order_id = p.order_id
WHERE o.order_status = 'canceled'
GROUP BY o.order_id, o.order_status;
"""

cancelled_paid_df = pd.read_sql(query, conn)
cancelled_paid_df.head()


,order_id,order_status,total_payment
0,00310b0c75bb13015ec4d82d341865a4,canceled,55.28
1,00ae7a8b4936674ebb701d4a23719a79,canceled,37.16
2,00b1cb0320190ca0daa2c88b35206009,canceled,0.00
3,00d0ffd14774da775ac832ba8520510f,canceled,134.49
4,00ff0cf5583758e6964723e42f111bf4,canceled,170.15


In [28]:
query = """
SELECT
    o.order_id,
    SUM(oi.price + oi.freight_value) AS order_value,
    julianday(o.order_delivered_customer_date) -
    julianday(o.order_estimated_delivery_date) AS delay_days
FROM orders o
JOIN order_items oi
ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY o.order_id
HAVING (
    julianday(o.order_delivered_customer_date) -
    julianday(o.order_estimated_delivery_date)
) > 5
ORDER BY order_value DESC;
"""

high_value_delayed_df = pd.read_sql(query, conn)
high_value_delayed_df.head()


,order_id,order_value,delay_days
0,fefacc66af859508bf1a7934eab1e97f,6922.21,5.623495
1,d7a2c0c1ff66b314f3bf166fb4157fd4,3184.55,23.981609
2,bf205457ee84ab84c423d67b88239982,2733.63,9.851308
3,a5d11ae16578088b0fcd76de30368bdb,2590.61,16.595486
4,6d0940a8f5fba47562bb14cd97dfd6da,2455.12,15.844190


In [29]:
query = """
SELECT
    order_id,
    order_status,
    order_purchase_timestamp,
    order_delivered_customer_date,
    order_estimated_delivery_date,
    julianday(order_delivered_customer_date) -
    julianday(order_estimated_delivery_date) AS delay_days
FROM orders;
"""

df = pd.read_sql(query, conn)

def classify_issue(row):
    if row["order_status"] == "canceled":
        return "CANCELLED"
    elif pd.notna(row["delay_days"]) and row["delay_days"] > 5:
        return "DELAYED"
    else:
        return "OK"

df["issue_type"] = df.apply(classify_issue, axis=1)

df.head()


,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delay_days,issue_type
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,-7.107488,OK
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,-5.355729,OK
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,-17.245498,OK
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,-12.980069,OK
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,-9.238171,OK


In [30]:
df.to_csv("order_issue_report.csv", index=False)


**API Validation**

In [31]:
# Select sample orders to simulate API requests
api_orders = orders[[
    "order_id",
    "order_status",
    "order_purchase_timestamp",
    "order_delivered_customer_date"
]].sample(200, random_state=42)

# Convert to JSON-like payloads
api_payload = api_orders.to_dict(orient="records")

api_payload[:2]


[{'order_id': 'b9a6c5f5df52c7226ac85aee7524c27f',
  'order_status': 'delivered',
  'order_purchase_timestamp': Timestamp('2018-06-12 20:07:44'),
  'order_delivered_customer_date': Timestamp('2018-06-19 12:44:08')},
 {'order_id': '261e71d2349c713eafa9f3df5972b95d',
  'order_status': 'delivered',
  'order_purchase_timestamp': Timestamp('2018-01-20 12:15:57'),
  'order_delivered_customer_date': Timestamp('2018-01-30 11:32:35')}]

In [32]:
VALID_STATUS = [
    "delivered", "shipped", "canceled",
    "processing", "invoiced", "unavailable"
]

def validate_order_api(order):
    errors = []

    if not order.get("order_id"):
        errors.append("Missing order_id")

    if order.get("order_status") not in VALID_STATUS:
        errors.append("Invalid order_status")

    if (
        order.get("order_status") == "delivered"
        and not order.get("order_delivered_customer_date")
    ):
        errors.append("Delivered order missing delivery date")

    return errors


In [33]:
api_test_results = []

for order in api_payload:
    issues = validate_order_api(order)

    api_test_results.append({
        "order_id": order["order_id"],
        "api_test_status": "FAIL" if issues else "PASS",
        "issues": "; ".join(issues)
    })

api_test_df = pd.DataFrame(api_test_results)
api_test_df.head()


,order_id,api_test_status,issues
0,b9a6c5f5df52c7226ac85aee7524c27f,PASS,
1,261e71d2349c713eafa9f3df5972b95d,PASS,
2,67b50899f52995848c427e361e10dde3,PASS,
3,32733fc014b67ef70fa6039dd8c6ba82,PASS,
4,39a70e9e9b729b11dee34ac12478597f,PASS,


In [34]:
api_test_df.to_sql(
    "api_validation_results",
    conn,
    index=False,
    if_exists="replace"
)


200

In [35]:
query = """
SELECT
    api_test_status,
    COUNT(*) AS total_orders
FROM api_validation_results
GROUP BY api_test_status;
"""

pd.read_sql(query, conn)


,api_test_status,total_orders
0,PASS,200


In [36]:
api_test_df.to_csv("api_validation_report.csv", index=False)
